In [3]:
# Enable all CPU cores
from tqdm import tqdm
import pandas as pd
import re
import pandas as pd

# Read the file content
def parse_to_datafame(file_path):
  with open(file_path, "r") as file:
      content = file.read()
  # Define a regex pattern to extract the test case and its details
  pattern = r" structure (?P<structure>.+)\s+nat\s+(?P<nat>\d+)\s+nprim\s+(?P<nprim>\d+)\s+maxl\s+(?P<maxl>\d+)\s+max_nnl\s+(?P<max_nnl>\d+)\s+mean_nnl\s+(?P<mean_nnl>\d+)\s+max_nsh\s+(?P<max_nsh>\d+)\s+cpu_time\s+(?P<cpu_time>[\d.]+)\s+gpu_gb_in\s+(?P<gpu_gb_in>[\d.]+)\s+gpu_gb_out\s+(?P<gpu_gb_out>[\d.]+)\s+gpu_gb_total\s+(?P<gpu_gb_total>[\d.]+)\s+gpu_between_atoms\s+(?P<gpu_between_atoms>[\d.]+)\s+gpu_in_atoms\s+(?P<gpu_in_atoms>[\d.]+)\s+gpu_time\s+(?P<gpu_time>[\d.]+)\s+gpu_walltime\s+(?P<gpu_walltime>[\d.]+)"

  # Use re.finditer to extract all matches
  matches = re.finditer(pattern, content)

  # Create a list of dictionaries to store the extracted data
  data = []
  for match in matches:
      data.append(match.groupdict())

  # Convert the list of dictionaries into a pandas DataFrame
  df = pd.DataFrame(data)

# Convert numeric columns to appropriate data types
  numeric_columns = [
      'nat', 'nprim', 'maxl', 'max_nnl', 'mean_nnl', 'max_nsh',
      'cpu_time', 'gpu_gb_in', 'gpu_gb_out', 'gpu_gb_total',
      'gpu_between_atoms', 'gpu_in_atoms', 'gpu_time', 'gpu_walltime'
  ]
  # Convert the numeric columns to float
  df[numeric_columns] = df[numeric_columns].apply(pd.to_numeric)

  # Display the DataFrame
  df = df.sort_values('nat')
  return df

option = 'h200'
!rm -rf notes/capture/{option}
!mkdir -p notes/capture/{option}
!rm -rf notes/data/{option}
!mkdir -p notes/data/{option}
for ncore in tqdm([1, 2, 4, 8, 16, 32]):
  ncore = str(ncore)
  !OMP_NUM_THREADS={ncore} meson test -C build/ hamiltonian --verbose -t 0 > notes/capture/{option}/{ncore}cpu.txt
  df = parse_to_datafame(f"notes/capture/{option}/{ncore}cpu.txt")
  df.to_csv(f"notes/data/{option}/{ncore}cpu.csv", index=False)

100%|██████████| 6/6 [04:50<00:00, 48.35s/it]


In [4]:
!nproc
!lscpu

192


Architecture:             x86_64
  CPU op-mode(s):         32-bit, 64-bit
  Address sizes:          46 bits physical, 57 bits virtual
  Byte Order:             Little Endian
CPU(s):                   192
  On-line CPU(s) list:    0-191
Vendor ID:                GenuineIntel
  Model name:             INTEL(R) XEON(R) PLATINUM 8568Y+
    CPU family:           6
    Model:                207
    Thread(s) per core:   2
    Core(s) per socket:   48
    Socket(s):            2
    Stepping:             2
    CPU max MHz:          4000.0000
    CPU min MHz:          800.0000
    BogoMIPS:             4600.00
    Flags:                fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge m
                          ca cmov pat pse36 clflush dts acpi mmx fxsr sse sse2 s
                          s ht tm pbe syscall nx pdpe1gb rdtscp lm constant_tsc 
                          art arch_perfmon pebs bts rep_good nopl xtopology nons
                          top_tsc cpuid aperfmperf tsc_known_freq 

In [5]:
!nvidia-smi

Wed May 21 08:08:31 2025       
+-----------------------------------------------------------------------------------------+
| NVIDIA-SMI 570.133.20             Driver Version: 570.133.20     CUDA Version: 12.8     |
|-----------------------------------------+------------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id          Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |           Memory-Usage | GPU-Util  Compute M. |
|                                         |                        |               MIG M. |
|=========================================+========================+======================|
|   0  NVIDIA H200                    On  |   00000000:4C:00.0 Off |                    0 |
| N/A   28C    P0             76W /  700W |       0MiB / 143771MiB |      0%      Default |
|                                         |                        |             Disabled |
+-----------------------------------------+-----